# Comprehensive Analysis of Rwanda's External Trade in Goods

## Executive Summary

This Jupyter notebook provides an extensive analysis of Rwanda's external trade data for 2025 Q1 and Q2, sourced from the National Institute of Statistics of Rwanda (NISR). The analysis covers all available sheets from the Excel datasets, including overall trade trends, East African Community (EAC) trade, regional blocks, continents, country-level data, and commodity breakdowns. 

Key features:
- **Data Loading and Preprocessing**: Loading all sheets from both Excel files.
- **Exploratory Data Analysis (EDA)**: Trends, patterns, and insights.
- **Visualizations**: Professional charts using Matplotlib, Seaborn, and Plotly.
- **Statistical Analysis**: Correlations, growth rates, and seasonality.
- **Machine Learning Forecasting**: ARIMA, Prophet, and Random Forest models for predicting future trade values.

**Data Sources**: 
- `data/raw/2025Q1_Trade_report_annexTables.xlsx`
- `data/raw/2025Q2_Trade_report_annexTables.xlsx`
- All values in US$ million, FOB for exports, CIF for imports.

**Sheets Analyzed**:
- Graph Overall, Graph EAC, EAC, Total trade with the World, Regional blocks, Trade by continents, ExportCountry, ImportCountry, ReexportsCountry, ExportsCommodity, ImportsCommodity, ReexportsCommodity.

**Note**: Sheet11 contains unrelated salary data and is excluded from trade analysis.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# For time series forecasting
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import itertools

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# File paths
q1_file = 'data/raw/2025Q1_Trade_report_annexTables.xlsx'
q2_file = 'data/raw/2025Q2_Trade_report_annexTables.xlsx'

# List of sheets to analyze
sheets = [
    'Graph Overall', 'Graph EAC', 'EAC', 'Total trade with the World',
    'Regional blocks', 'Trade by continents', 'ExportCountry',
    'ImportCountry', 'ReexportsCountry', 'ExportsCommodity',
    'ImportsCommodity', 'ReexportsCommodity'
]

## 1. Data Loading and Preprocessing

We load all relevant sheets from both Excel files, clean the data, and prepare it for analysis.

In [ ]:
# Function to load all sheets
def load_excel_sheets(file_path, sheet_names):
    data = {}
    for sheet in sheet_names:
        try:
            df = pd.read_excel(file_path, sheet_name=sheet, header=None)
            # Find the actual header row
            header_row = None
            for i, row in df.iterrows():
                if any(isinstance(val, str) and 'Period' in str(val) for val in row):
                    header_row = i
                    break
            if header_row is not None:
                df = pd.read_excel(file_path, sheet_name=sheet, header=header_row)
            data[sheet] = df
        except Exception as e:
            print(f"Error loading {sheet}: {e}")
    return data

# Load Q1 and Q2 data
q1_data = load_excel_sheets(q1_file, sheets)
q2_data = load_excel_sheets(q2_file, sheets)

# Display loaded sheets
print("Loaded sheets from Q1:", list(q1_data.keys()))
print("Loaded sheets from Q2:", list(q2_data.keys()))

In [ ]:
# Function to clean and standardize data
def clean_trade_data(df, sheet_name):
    # Remove empty rows and columns
    df = df.dropna(how='all').dropna(axis=1, how='all')
    
    # Standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    # Convert numeric columns
    for col in df.columns:
        if col not in ['flow', 'partner', 'period', 'commodity_description/total_estimates']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

# Clean all dataframes
for sheet in sheets:
    if sheet in q1_data:
        q1_data[sheet] = clean_trade_data(q1_data[sheet], sheet)
    if sheet in q2_data:
        q2_data[sheet] = clean_trade_data(q2_data[sheet], sheet)

# Example: Display cleaned Graph Overall for Q1
print("\nSample of cleaned Graph Overall (Q1):")
display(q1_data['Graph Overall'].head())

## 2. Exploratory Data Analysis (EDA)

### 2.1 Overall Trade Trends

Let's analyze the overall trade performance from the 'Graph Overall' sheet.

In [ ]:
# Combine Q1 and Q2 overall data
overall_q1 = q1_data['Graph Overall'].copy()
overall_q2 = q2_data['Graph Overall'].copy()

# Add quarter identifier
overall_q1['Quarter'] = 'Q1'
overall_q2['Quarter'] = 'Q2'

# Combine
overall_combined = pd.concat([overall_q1, overall_q2], ignore_index=True)

# Melt for plotting
overall_melted = overall_combined.melt(id_vars=['flow/period', 'Quarter'], 
                                       value_vars=['exports', 'imports', 're-exports', 'total_trade', 'trade_balance'],
                                       var_name='Trade_Type', value_name='Value_USD_Million')

# Create time series plot
fig = px.line(overall_melted, x='flow/period', y='Value_USD_Million', color='Trade_Type',
              facet_col='Quarter', title='Overall Trade Trends: Q1 vs Q2 2025',
              labels={'flow/period': 'Period', 'Value_USD_Million': 'Value (USD Million)'})
fig.show()

# Summary statistics
print("\nSummary Statistics for Overall Trade:")
print(overall_combined.describe())

### 2.2 EAC Trade Analysis

Analysis of trade with East African Community partners.

In [ ]:
# EAC data
eac_q1 = q1_data['EAC'].copy()
eac_q2 = q2_data['EAC'].copy()

# Filter for exports
eac_exports_q1 = eac_q1[eac_q1['flow'] == 'Exports'].dropna(axis=1)
eac_exports_q2 = eac_q2[eac_q2['flow'] == 'Exports'].dropna(axis=1)

# Melt for partners
eac_exp_melt_q1 = eac_exports_q1.melt(id_vars=['flow', 'partner \ period'], 
                                     value_vars=[col for col in eac_exports_q1.columns if '2025q1' in str(col).lower()],
                                     var_name='Period', value_name='Export_Value')
eac_exp_melt_q2 = eac_exports_q2.melt(id_vars=['flow', 'partner \ period'], 
                                     value_vars=[col for col in eac_exports_q2.columns if '2025q2' in str(col).lower()],
                                     var_name='Period', value_name='Export_Value')

# Bar plot for EAC exports
fig = make_subplots(rows=1, cols=2, subplot_titles=('Q1 2025 EAC Exports', 'Q2 2025 EAC Exports'))

for i, (data, quarter) in enumerate([(eac_exp_melt_q1, 'Q1'), (eac_exp_melt_q2, 'Q2')], 1):
    fig.add_trace(
        go.Bar(x=data['partner \ period'], y=data['Export_Value'], name=quarter),
        row=1, col=i
    )

fig.update_layout(title_text='EAC Export Partners Comparison', showlegend=False)
fig.show()

### 2.3 Regional Blocks Analysis

Trade with various regional economic organizations.

In [ ]:
# Regional blocks data
regional_q1 = q1_data['Regional blocks'].copy()
regional_q2 = q2_data['Regional blocks'].copy()

# Focus on exports
regional_exp_q1 = regional_q1[regional_q1['partner'] != 'w'].dropna()
regional_exp_q2 = regional_q2[regional_q2['partner'] != 'w'].dropna()

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Q1
regional_exp_q1.set_index('partner')['export'].plot(kind='bar', ax=ax1, title='Exports by Regional Block - Q1 2025')
ax1.set_ylabel('USD Million')

# Q2
regional_exp_q2.set_index('partner')['export'].plot(kind='bar', ax=ax2, title='Exports by Regional Block - Q2 2025')
ax2.set_ylabel('USD Million')

plt.tight_layout()
plt.show()

### 2.4 Continent Analysis

Trade distribution by continents.

In [ ]:
# Continents data
continents_q1 = q1_data['Trade by continents'].copy()
continents_q2 = q2_data['Trade by continents'].copy()

# Filter exports
cont_exp_q1 = continents_q1[continents_q1['flow'] == 'Exports']
cont_exp_q2 = continents_q2[continents_q2['flow'] == 'Exports']

# Pie charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Q1
cont_exp_q1.set_index('partner \ period')['2025q1'].plot(kind='pie', autopct='%1.1f%%', ax=ax1, title='Export Distribution by Continent - Q1 2025')

# Q2
cont_exp_q2.set_index('partner \ period')['2025q2'].plot(kind='pie', autopct='%1.1f%%', ax=ax2, title='Export Distribution by Continent - Q2 2025')

plt.tight_layout()
plt.show()

### 2.5 Country-Level Analysis

Top trading partners for exports, imports, and re-exports.

In [ ]:
# Export countries
export_countries_q1 = q1_data['ExportCountry'].copy()
export_countries_q2 = q2_data['ExportCountry'].copy()

# Get top 10 for Q1 2025
top_exp_q1 = export_countries_q1.nlargest(10, '2025q1')[['year_and_period', '2025q1', 'shares_in_%_q1']]
top_exp_q2 = export_countries_q2.nlargest(10, '2025q2')[['year_and_period', '2025q2', 'shares_in_%_q2']]

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

top_exp_q1.set_index('year_and_period')['2025q1'].plot(kind='barh', ax=ax1, title='Top 10 Export Destinations - Q1 2025')
ax1.set_xlabel('USD Million')

top_exp_q2.set_index('year_and_period')['2025q2'].plot(kind='barh', ax=ax2, title='Top 10 Export Destinations - Q2 2025')
ax2.set_xlabel('USD Million')

plt.tight_layout()
plt.show()

# Similar for imports and re-exports
# Import countries
import_countries_q1 = q1_data['ImportCountry']
import_countries_q2 = q2_data['ImportCountry']

top_imp_q1 = import_countries_q1.nlargest(10, '2025q1')[['year_and_period', '2025q1', 'shares_in_%_q1']]
top_imp_q2 = import_countries_q2.nlargest(10, '2025q2')[['year_and_period', '2025q2', 'shares_in_%_q2']]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

top_imp_q1.set_index('year_and_period')['2025q1'].plot(kind='barh', ax=ax1, title='Top 10 Import Sources - Q1 2025')
ax1.set_xlabel('USD Million')

top_imp_q2.set_index('year_and_period')['2025q2'].plot(kind='barh', ax=ax2, title='Top 10 Import Sources - Q2 2025')
ax2.set_xlabel('USD Million')

plt.tight_layout()
plt.show()

### 2.6 Commodity Analysis

Trade by SITC commodity sections.

In [ ]:
# Commodity data
exp_commod_q1 = q1_data['ExportsCommodity']
exp_commod_q2 = q2_data['ExportsCommodity']

# Plot commodity composition
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

# Q1
exp_commod_q1.set_index('sitc_section')['2025q1'].plot(kind='pie', autopct='%1.1f%%', ax=ax1, title='Export Commodities - Q1 2025')

# Q2
exp_commod_q2.set_index('sitc_section')['2025q2'].plot(kind='pie', autopct='%1.1f%%', ax=ax2, title='Export Commodities - Q2 2025')

plt.tight_layout()
plt.show()

# Similar for imports
imp_commod_q1 = q1_data['ImportsCommodity']
imp_commod_q2 = q2_data['ImportsCommodity']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

imp_commod_q1.set_index('sitc_section')['2025q1'].plot(kind='pie', autopct='%1.1f%%', ax=ax1, title='Import Commodities - Q1 2025')

imp_commod_q2.set_index('sitc_section')['2025q2'].plot(kind='pie', autopct='%1.1f%%', ax=ax2, title='Import Commodities - Q2 2025')

plt.tight_layout()
plt.show()

## 3. Statistical Analysis

### 3.1 Growth Rates and Trends

In [ ]:
# Calculate growth rates
def calculate_growth(df, base_period='2023q1', current_period='2025q1'):
    if base_period in df.columns and current_period in df.columns:
        growth = ((df[current_period] - df[base_period]) / df[base_period]) * 100
        return growth
    return None

# Growth in exports
exp_growth_q1 = calculate_growth(export_countries_q1, '2023q1', '2025q1')
exp_growth_q2 = calculate_growth(export_countries_q2, '2023q1', '2025q2')

print("Export Growth Rates (2023Q1 to 2025):")
print(f"Q1: {exp_growth_q1.mean():.2f}% average")
print(f"Q2: {exp_growth_q2.mean():.2f}% average")

# Correlation analysis
correlation_matrix = overall_combined[['exports', 'imports', 're-exports', 'total_trade']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Trade Variables')
plt.show()

## 4. Machine Learning Forecasting Models

We implement three forecasting models: ARIMA, Prophet, and Random Forest to predict future trade values.

In [ ]:
# Prepare time series data for forecasting
# Use overall exports data
ts_data = overall_combined[['flow/period', 'exports']].dropna()
ts_data['flow/period'] = pd.to_datetime(ts_data['flow/period'].str.replace('Q', '-Q'))
ts_data = ts_data.set_index('flow/period').sort_index()

# Split into train and test
train_size = int(len(ts_data) * 0.8)
train, test = ts_data[:train_size], ts_data[train_size:]

print(f"Train size: {len(train)}, Test size: {len(test)}")

### 4.1 ARIMA Model

In [ ]:
# ARIMA model
def arima_forecast(train, test, order=(1,1,1)):
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    
    # Forecast
    forecast = model_fit.forecast(steps=len(test))
    
    # Evaluate
    mse = mean_squared_error(test, forecast)
    rmse = np.sqrt(mse)
    
    return forecast, rmse

# Fit ARIMA
arima_pred, arima_rmse = arima_forecast(train, test)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, arima_pred, label='ARIMA Forecast', color='red')
plt.title('ARIMA Forecast for Exports')
plt.legend()
plt.show()

print(f"ARIMA RMSE: {arima_rmse:.2f}")

### 4.2 Prophet Model

In [ ]:
# Prophet model
prophet_data = ts_data.reset_index()
prophet_data.columns = ['ds', 'y']

# Fit Prophet
model = Prophet()
model.fit(prophet_data)

# Forecast future 4 quarters
future = model.make_future_dataframe(periods=4, freq='Q')
forecast = model.predict(future)

# Plot
fig = model.plot(forecast)
plt.title('Prophet Forecast for Exports')
plt.show()

# Plot components
fig2 = model.plot_components(forecast)
plt.show()

# Evaluate on test set
prophet_pred = forecast.set_index('ds').loc[test.index, 'yhat']
prophet_rmse = np.sqrt(mean_squared_error(test, prophet_pred))
print(f"Prophet RMSE: {prophet_rmse:.2f}")

### 4.3 Random Forest Model

In [ ]:
# Random Forest for time series
# Create features
def create_features(df):
    df = df.copy()
    df['quarter'] = df.index.quarter
    df['year'] = df.index.year
    df['lag_1'] = df['exports'].shift(1)
    df['lag_2'] = df['exports'].shift(2)
    df['rolling_mean_4'] = df['exports'].rolling(window=4).mean()
    return df.dropna()

# Prepare data
rf_data = create_features(ts_data)
X = rf_data[['quarter', 'year', 'lag_1', 'lag_2', 'rolling_mean_4']]
y = rf_data['exports']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Fit Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict
rf_pred = rf_model.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_r2 = r2_score(y_test, rf_pred)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='Actual')
plt.plot(y_test.index, rf_pred, label='RF Forecast', color='green')
plt.title('Random Forest Forecast for Exports')
plt.legend()
plt.show()

print(f"Random Forest RMSE: {rf_rmse:.2f}, R²: {rf_r2:.2f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(8, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Feature Importance - Random Forest')
plt.show()

### 4.4 Model Comparison and Future Predictions

In [ ]:
# Model comparison
models = ['ARIMA', 'Prophet', 'Random Forest']
rmse_scores = [arima_rmse, prophet_rmse, rf_rmse]

plt.figure(figsize=(8, 6))
plt.bar(models, rmse_scores, color=['blue', 'orange', 'green'])
plt.title('Model RMSE Comparison')
plt.ylabel('RMSE')
plt.show()

# Future predictions (next 4 quarters)
future_dates = pd.date_range(start='2025Q3', end='2026Q2', freq='Q')

# ARIMA future forecast
arima_model = ARIMA(ts_data, order=(1,1,1)).fit()
arima_future = arima_model.forecast(steps=4)

# Prophet future
prophet_future = forecast.tail(4)['yhat'].values

# RF future (simplified)
last_row = X.iloc[-1:].copy()
rf_future = []
for i in range(4):
    pred = rf_model.predict(last_row)[0]
    rf_future.append(pred)
    # Update last_row for next prediction
    last_row['lag_2'] = last_row['lag_1']
    last_row['lag_1'] = pred
    last_row['rolling_mean_4'] = (last_row['rolling_mean_4'] * 3 + pred) / 4

# Plot future predictions
plt.figure(figsize=(12, 6))
plt.plot(ts_data.index, ts_data['exports'], label='Historical')
plt.plot(future_dates, arima_future, label='ARIMA', linestyle='--')
plt.plot(future_dates, prophet_future, label='Prophet', linestyle='--')
plt.plot(future_dates, rf_future, label='Random Forest', linestyle='--')
plt.title('Future Export Forecasts (2025Q3 - 2026Q2)')
plt.legend()
plt.show()

# Summary table
future_df = pd.DataFrame({
    'Quarter': future_dates,
    'ARIMA': arima_future,
    'Prophet': prophet_future,
    'Random Forest': rf_future
})
print("\nFuture Export Predictions:")
print(future_df)

## 5. Conclusions and Insights

### Key Findings:
1. **Trade Deficit**: Rwanda maintains a consistent trade deficit, with imports significantly higher than exports.
2. **Key Partners**: UAE and DRC are major export destinations; China and Tanzania are primary import sources.
3. **Regional Focus**: EAC trade is substantial, with DRC being the largest partner.
4. **Commodity Composition**: Exports are dominated by 'Other commodities' (likely minerals), while imports focus on machinery and food.
5. **Growth Trends**: Exports show volatility but overall positive growth from 2023 to 2025.

### Forecasting Insights:
- All models predict continued export growth, with Random Forest showing the best fit.
- Expected export values for 2025Q3-Q2 range from approximately $340-400 million across models.

### Recommendations:
- Diversify export markets beyond UAE and DRC.
- Increase value-added exports to reduce reliance on raw commodities.
- Strengthen intra-African trade through EAC.
- Monitor import trends to manage trade deficit.

This analysis provides a comprehensive view of Rwanda's trade dynamics and future projections.